In [0]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
df = pd.merge(df1, df2, on='pj_no', how='outer')

del df1
del df2

dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')

del dftest1
del dftest2

df = df.drop('keiyaku_pr', axis=1)

print(df.shape)
print(dftest.shape)

for i, j in zip(df.columns, dftest.columns):
  if i != j:
    print(i)
    print(j)

(6461, 157)
(4273, 157)


In [4]:
# 欠損値を回帰代入法により埋めていく

# dfとdftestの欠損値が異なる列にも存在するので、その分を予測用に確保
#df_preds = pd.DataFrame(df['road1_sb'])
#df_preds = pd.concat(objs=[df_preds, df['gk_yoc_tm']], axis=1)
#df = df.drop('road1_sb', axis=1)
#df = df.drop('gk_yoc_tm', axis=1)

#dftest_preds = pd.DataFrame(dftest['chiseki_kb_hb'])
#dftest_preds = pd.concat(objs=[dftest_preds, dftest['tt_mseki_avg_hb']], axis=1)
#dftest = dftest.drop('chiseki_kb_hb', axis=1)
#dftest = dftest.drop('tt_mseki_avg_hb', axis=1)

df_preds = pd.DataFrame()
dftest_preds = pd.DataFrame()

# dfとdftestの欠損値が含まれる列を予測用に確保（上下同じ列）
for i, j in zip(df.isnull().any(), df.columns):  
    if i == True or j == 'road1_sb' or j == 'gk_yoc_tm':
        df_preds = pd.DataFrame(pd.concat(objs=[df_preds, df[j]], axis=1))
        df = df.drop(j, axis=1)

for i, j in zip(dftest.isnull().any(), dftest.columns):
    if i == True or j == 'chiseki_kb_hb' or j == 'tt_mseki_avg_hb':
        dftest_preds = pd.concat(objs=[dftest_preds, dftest[j]], axis=1)
        dftest = dftest.drop(j, axis=1)
        
print(df.shape)
print(dftest.shape)
print(df_preds.shape)
print(dftest_preds.shape)
df_preds.head()

(6461, 53)
(4273, 53)
(6461, 104)
(4273, 104)


,chiseki_kb_hb,yoto2,kempei2,yoseki2,tateuri_su,tochiuri_su,joken_su,hy1f_date_su,hy2f_date_su,hy3f_date_su,road1_sb,road2_hk,road2_sb,road2_fi,road2_mg,road3_hk,road3_sb,road3_fi,road3_mg,road4_hk,road4_sb,road4_fi,road4_mg,kaoku_hb,toshikuiki2,minmenseki,hokakisei1,hokakisei2,hokakisei3,hokakisei4,mseki_rd_hb,mseki_dp_hb,tt_mseki_avg_hb,fi4m_kyori,fi3m_kyori,bus_hon,sho_conv,sho_super,sho_shoten,sho_market,...,rs_w_kdate3,rs_w_parking,rs_w_zoki,rs_w_m_ari,rs_w_m_nashi,rs_w_tahata,rs_s_kdate2,rs_s_kdate3,rs_s_parking,rs_s_zoki,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari,kobetsu1,kobetsu2,kobetsu3,kobetsu4
0,109.10,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,私道,東,私道,4.0,10.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.00,104.43,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,9.9,東,良い,整形地,普通,床暖房付,NaN,NaN,NaN
1,136.00,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,公道,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,農地法,NaN,NaN,0.0,0.00,105.16,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.2,問題なし,9.3,北西,普通,不整形地,普通,NaN,NaN,NaN,NaN
2,86.52,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,私道,北,公道,3.8,5.2,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,62.1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,85.96,0.0,0.0,35.0,○,NaN,NaN,NaN,...,NaN,○,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,11.1,東,普通,整形地,普通,NaN,NaN,NaN,NaN
3,164.07,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,公道,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.84,105.99,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,10.5,南,普通,整形地,良い,NaN,NaN,NaN,NaN
4,403.58,NaN,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,私道,北東,私道,4.0,13.9,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,埋蔵文化財,NaN,0.0,0.00,88.73,0.0,0.0,NaN,○,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,13.5,北,良い,整形地,普通,床暖房付,NaN,NaN,NaN


In [5]:
df.head()

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage
0,0,埼玉県朝霞市泉水,109.26,工業地域,60,200,公営,公共下水,個別プロパン,宅内処理,北,4.0,7.0,（無）,0.6,（無）,（不要）,800,1000,実測,更地,市街化区域,第2種,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,135000,188000,197000,109.26,109.26,109.26,104.43,104.43,109.26,（不要）,（不要）,（不要）,10,13,東武東上線,朝霞台,徒歩,17,train_0000,109.26,104.43,4.0,1
1,1,埼玉県川越市大字的場,136.15,第一種低層住居専用地域,60,200,公営,公共下水,個別プロパン,宅内処理,北西,4.2,9.3,（無）,0.0,（無）,（不要）,1680,1840,実測,更地,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,22条区域,（無）,68000,84900,48600,136.15,136.15,136.15,109.10,109.10,136.15,（不要）,（不要）,（不要）,21,23,JR川越線,的場,徒歩,13,train_0001,136.11,105.16,3.0,1
2,2,埼玉県川口市安行原,86.92,第一種低層住居専用地域,50,100,私営,個別浄化槽,個別プロパン,宅内処理,東,4.0,11.1,（有）,0.2,（無）,（不要）,600,550,実測,建物有,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,83000,116000,124000,86.92,86.92,86.92,85.96,85.96,86.92,（不要）,（不要）,（要）,8,7,埼玉高速鉄道線,新井宿,徒歩,27,train_0002,87.07,85.94,2.8,1
3,3,埼玉県草加市松江,164.91,工業地域,60,200,公営,公共下水,都市ガス,宅内処理,南,4.0,13.3,（無）,0.3,（無）,（不要）,1220,111,公簿,更地,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（要）,防火指定無,（無）,91000,122000,113000,164.07,164.07,164.07,105.99,105.99,164.07,（不要）,（不要）,（不要）,16,2,東武伊勢崎線,獨協大学前駅〈草加松原〉,徒歩,18,train_0003,163.75,105.98,4.0,2
4,4,埼玉県新座市野火止,403.58,第一種低層住居専用地域,50,80,公営,公共下水,都市ガス,宅内処理,南西,4.0,13.9,（無）,0.1,（無）,（不要）,480,2800,公簿,更地,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,22条区域,（無）,160000,179000,160000,334.58,111.52,111.53,89.27,89.27,111.52,（不要）,（不要）,（不要）,6,35,JR武蔵野線,新座,徒歩,7,train_0004,111.51,89.01,2.0,2


In [6]:
dftest.head()

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage
0,2781,埼玉県羽生市東,208.26,第一種低層住居専用地域,50,80,公営,公共下水,個別プロパン,宅内処理,西,4.0,13.0,（無）,0.4,（無）,（不要）,750,1700,公簿,建物有,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,21000,29000,33200,208.26,208.26,208.26,115.09,115.09,208.26,（不要）,（不要）,（不要）,10,22,東武伊勢崎線,羽生,徒歩,21,test_0000,212.38,118.40,6.5,2
1,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.1,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,（不要）,（不要）,13,10,東武東上線,新河岸,徒歩,11,test_0001,100.87,97.08,1.6,1
2,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.1,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,（不要）,（不要）,13,10,東武東上線,新河岸,徒歩,11,test_0002,106.02,97.19,2.0,1
3,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.1,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,（不要）,（不要）,13,10,東武東上線,新河岸,徒歩,11,test_0003,110.15,100.60,2.1,1
4,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.1,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,（不要）,（不要）,13,10,東武東上線,新河岸,徒歩,11,test_0004,112.13,97.29,2.1,2


In [0]:
for i, j in zip(df.columns, dftest.columns):
  if i != j:
    print(i)
    print(j)

In [8]:
df = pd.concat(objs=[df, df_preds], axis=1)
dftest = pd.concat(objs=[dftest, dftest_preds], axis=1)
df_train = pd.DataFrame(df)
#df_test = pd.DataFrame(df)
dftest_train = pd.DataFrame(dftest)
#dftest_test = pd.DataFrame(dftest)
for i, j in zip(dftest_train.dtypes, dftest_train.columns):
    if j == 'bas_toho1':
        print(i)
        print(j)
dftest_train

object
bas_toho1


,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,...,rs_w_kdate3,rs_w_parking,rs_w_zoki,rs_w_m_ari,rs_w_m_nashi,rs_w_tahata,rs_s_kdate2,rs_s_kdate3,rs_s_parking,rs_s_zoki,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari,kobetsu1,kobetsu2,kobetsu3,kobetsu4
0,2781,埼玉県羽生市東,208.26,第一種低層住居専用地域,50,80,公営,公共下水,個別プロパン,宅内処理,西,4.0,13.0,（無）,0.40,（無）,（不要）,750,1700,公簿,建物有,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,21000,29000,33200,208.26,208.26,208.26,115.09,115.09,208.26,（不要）,...,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,徒歩,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,13.00,西,良い,整形地,良い,行き止まり途中,NaN,NaN,NaN
1,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,7.90,南,良い,整形地,良い,二方路,NaN,NaN,NaN
2,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,7.10,西,悪い,整形地,普通,NaN,NaN,NaN,NaN
3,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,9.50,南西,良い,整形地,良い,角地,二方路,NaN,NaN
4,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,14.10,北西,良い,整形地,普通,角地,ごみ置き場前,NaN,NaN
5,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,7.30,北,良い,整形地,悪い,NaN,NaN,NaN,NaN
6,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,14.40,北西,良い,整形地,普通,角地,NaN,NaN,NaN
7,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,83000,130000,130000,1094.62,100.10,120.53,95.90,95.90,109.46,（不要）,...,NaN,NaN,NaN,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,15.00,北,良い,整形地,悪い,NaN,NaN,NaN,NaN
8,2782,埼玉県川越市大字寺尾,1369.35,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,西,1.8,25.9,（無）,0.10,（無）,（不要）,1000,800,実測,更地,市街化区域,指定無,（無）,（無）,（要）,（不要）,（不要）,22条区域,（無）,8300

In [9]:
preds_name = 'road1_sb'

df_test_num = []
dftest_test_num = []

for line_num, judg in enumerate(df[preds_name].isnull()):
    if judg == True: # 欠損値あり
        df_train = df_train.drop(line_num, axis=0)
        df_test_num.append(line_num)
    #else: # 欠損値なし
        #df_test = df_test.drop(line_num, axis=0)

for line_num, judg in enumerate(dftest[preds_name].isnull()):
    if judg == True: # 欠損値あり
        dftest_train = dftest_train.drop(line_num, axis=0)
        dftest_test_num.append(line_num)
    #else: # 欠損値なし
        #dftest_test = dftest_test.drop(line_num, axis=0)
        
print(df.shape)
print(dftest.shape)
print(df_train.shape)
print(dftest_train.shape)
print(df_train[preds_name].isnull().any())
print(dftest_train[preds_name].isnull().any())
#print(dftest_train.loc[4272, preds_name])
print(len(dftest_train))
df_train = df_train.reset_index(drop=True)
dftest_train = dftest_train.reset_index(drop=True)
#df_test = df_test.reset_index(drop=True)
#dftest_test = dftest_test.reset_index(drop=True)

(6461, 157)
(4273, 157)
(6461, 157)
(4272, 157)
False
False
4272


In [10]:
dftest_train.loc[1337:1339, :]

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,...,rs_w_kdate3,rs_w_parking,rs_w_zoki,rs_w_m_ari,rs_w_m_nashi,rs_w_tahata,rs_s_kdate2,rs_s_kdate3,rs_s_parking,rs_s_zoki,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari,kobetsu1,kobetsu2,kobetsu3,kobetsu4
1337,3366,埼玉県加須市陽光台,1170.68,指定のない区域,60,200,公営,公共下水,都市ガス,宅内処理,北,6.0,11.2,（無）,0.3,（無）,（不要）,1000,2200,公簿,造成上り,非線引き区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,26800,19100,35300,1170.68,164.83,170.47,106.00,113.00,167.24,（不要）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,6.0,問題なし,13.8,南,良い,整形地,良い,眺望良,NaN,NaN,NaN
1338,3368,埼玉県さいたま市見沼区大字大谷,100.03,第一種低層住居専用地域,50,100,公営,公共下水,都市ガス,宅内処理,南西,4.5,9.4,（無）,0.8,（無）,（不要）,600,750,公簿,建物有,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（不要）,防火指定無,（無）,81000,105000,108000,100.03,100.03,100.03,94.77,94.77,100.03,（不要）,...,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.8,歩道あり,9.4,南西,良い,整形地,普通,NaN,NaN,NaN,NaN
1339,3369,埼玉県越谷市弥栄町,173.40,第一種低層住居専用地域,60,100,公営,公共下水,都市ガス,宅内処理,北,4.0,11.6,（無）,0.3,（無）,（不要）,1540,930,実測,更地,市街化区域,指定無,（無）,（無）,（不要）,（不要）,（要）,22条区域,（無）,74000,91000,91000,173.40,173.40,173.40,109.12,109.12,173.40,（不要）,...,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,11.5,北,良い,整形地,普通,NaN,NaN,NaN,NaN


In [11]:
# 間違っているセル（2019/06/26）
if len(df_train.loc[:, :preds_name]) != 0:
    # df_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(df_train.loc[:, :preds_name].columns, df_train.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = df_train[i].factorize()
        id_3 = pd.DataFrame({i: id_1})
        df_train[i] = id_3
    del factorize

if len(dftest_train.loc[:, :preds_name]) != 0:
    # dftest_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(dftest_train.loc[:, :preds_name].columns, dftest_train.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
        #print(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = dftest_train[i].factorize()
        #print(len(id_1))
        id_3 = pd.DataFrame({i: id_1})
        dftest_train[i] = id_3
    
id_3
    #del factorize

,road1_sb
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [12]:
print(len(dftest_train.loc[:, :preds_name]))
#dftest_train[preds_name]
#dftest_train.iloc[:, :64].head()
#print(dftest_train.columns.get_loc(preds_name))
#df_train = df_train.astype(int)
#dftest_train = dftest_train.astype(int)
#for i in df_train.loc[:, :preds_name].columns:    
#df_train.loc[:, :preds_name].dtypes
#df_train = df_train.astype(int)

4272


In [13]:
print(df_train[preds_name].isnull().any())
print(dftest_train[preds_name].isnull().any())
for i, j in enumerate(dftest_train.loc[:, :preds_name].isnull()):
    if j == True:
        print(i)
#print(dftest_train.loc[4272, preds_name])
#print(df_train.iloc[:, :df_train.columns.get_loc(preds_name)].isnull().any())
#print(dftest.iloc[:, :df_train.columns.get_loc(preds_name)].isnull().any())
#print(len(df_train))
#print(len(dftest_train))
#print(len(df_test))
#print(len(dftest_test))
#print(df_train.loc[0, preds_name])
#df_train.iloc[:, :df_train.columns.get_loc(preds_name)].head()
#dftest_train.iloc[:, :df_train.columns.get_loc(preds_name)].dtypes
#for i, j in zip(dftest_train.iloc[:, :df_train.columns.get_loc(preds_name)].dtypes, dftest_train.iloc[:, :df_train.columns.get_loc(preds_name)].columns):
#    if i == 'object':
#        print(i)
#        print(j)
#dftest_train.loc[4272, preds_name] = 1.0
#for i, j in enumerate(dftest_train.iloc[:, :df_train.columns.get_loc(preds_name)].isnull()):
#    if j == True:
#        print(j)
#        print(i)
#dftest_train.iloc[:, :df_train.columns.get_loc(preds_name)].isnull().any()

False
False


In [0]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor()

# :preds_name は preds_name が入らない
# preds_name: は preds_name が入る
X = pd.DataFrame(pd.concat(objs=[df_train.iloc[:, :df_train.columns.get_loc(preds_name)], dftest_train.iloc[:, :dftest_train.columns.get_loc(preds_name)]], axis=0))
T = pd.DataFrame(pd.concat(objs=[df_train[preds_name], dftest_train[preds_name]], axis=0))
#clf.fit(X, T)

#if len(df_test_num) != 0:
#    for preds_line in df_test_num:
#        df.loc[preds_line, preds_name] = clf.predict(df.loc[df_test_num, preds_name])
    
#if len(dftest_x_test) != 0:
#    for preds_line in dftest_test_num:
#        dftest.loc[preds_line, preds_name] = clf.predict(dftest.loc[dftest_test_num, preds_name])

In [15]:
X.iloc[:, :df_train.columns.get_loc(preds_name)].isnull().any()

pj_no              False
jukyo              False
chiseki_js_hb      False
yoto1              False
kempei1            False
yoseki1            False
josui              False
gesui              False
gas                False
usui               False
road1_hk           False
road1_fi           False
road1_mg           False
kaoku_um           False
yheki_kotei        False
yheki_umu          False
yheki_yohi         False
gk_sho_kyori       False
gk_chu_kyori       False
kborjs             False
hw_status          False
toshikuiki1        False
kodochiku          False
chikukeikaku       False
keikakuroad        False
kaihatsukyoka      False
t53kyoka           False
hokakyoka          False
bokachiiki         False
kinshijiko         False
                   ...  
mseki_yt_hb        False
tc_mseki_min_hb    False
tc_mseki_max_hb    False
tt_mseki_min_hb    False
tt_mseki_max_hb    False
tc_mseki_avg_hb    False
fi4m_yohi          False
fi3m_yohi          False
bus_yohi           False


In [0]:
for i, j in zip(df_train.loc[:, :preds_name], dftest_train.loc[:, :preds_name]):
  if i != j:
    print(i)
    print(j)

In [0]:
df_train.loc[:, :preds_name].head()

In [0]:
dftest_train.loc[:, :preds_name].head()